In [1]:
import os

import keras
import numpy
from keras import backend as K
from keras.datasets import fashion_mnist
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential, model_from_json

WORKING_DIR = %pwd

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [3]:
batch_size = 128
num_classes = 10 # because there are 10 different types of clothes
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [4]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()


In [5]:
x_train.shape

(60000, 28, 28)

In [6]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [8]:
weights_path = os.path.join(WORKING_DIR, 'weights.h5')
model_path = os.path.join(WORKING_DIR, 'model.json')
history_path = os.path.join(WORKING_DIR, 'history.csv')
print(os.path.exists(weights_path))

False


In [7]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger(history_path, append=True, separator=';')

def create_model(x_train, y_train, x_test, y_test, **kwargs):
    batch_size = kwargs.get('batch_size')
    epochs = kwargs.get('epochs')
    num_classes = kwargs.get('num_classes')

    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test), 
                        callbacks=[csv_logger])
    return model


In [9]:
if os.path.exists(weights_path) and os.path.exists(model_path):
    # load json and create model
    with open(model_path, 'r') as json_file:
        loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(weights_path)
    print("Loaded model from disk")
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
else:
    model = create_model(x_train, y_train, x_test, y_test, batch_size=batch_size,
                                  epochs=epochs, num_classes=num_classes)
    # serialize model to JSON
    model_json = model.to_json()
    with open(model_path, "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(weights_path)
    print("Saved model to disk")

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Train on 60000 samples, validate on 10000 samples
Epoch 1/12


  128/60000 [..............................] - ETA: 5:52 - loss: 2.2965 - acc: 0.0703

  256/60000 [..............................] - ETA: 3:36 - loss: 2.2502 - acc: 0.1406

  384/60000 [..............................] - ETA: 2:50 - loss: 2.2231 - acc: 0.1693

  512/60000 [..............................] - ETA: 2:26 - loss: 2.1915 - acc: 0.1914

...

59776/60000 [============================>.] - ETA: 0s - loss: 0.2143 - acc: 0.9227

59904/60000 [============================>.] - ETA: 0s - loss: 0.2143 - acc: 0.9227

60000/60000 [==============================] - 76s 1ms/step - loss: 0.2142 - acc: 0.9228 - val_loss: 0.2378 - val_acc: 0.9162


Saved model to disk


TypeError: can't pickle _thread.lock objects

In [ ]:
import pandas as pd

history = pd.read_csv(history_path, delimiter=';')